# SDG 6.6.1: Analysis and Reporting

## Setup

In [ ]:
# Install required geospatial libraries
!pip3 install gdal rtree geopandas rasterio

In [31]:
from datetime import datetime
import os

import altair as alt
import ee
import geopandas as gpd
from google.colab import drive
import pandas as pd
from shapely.ops import unary_union

In [ ]:
# Mount your Google Drive if not already done so
# You may see a pop-up window in which to confirm access.
drive.mount('drive')
drive_home = '/content/drive/MyDrive'

In [5]:
# Set name of directory where repo is located. By default 'sdg_661_analysis_and_reporting'
repo_dir_name = 'sdg_661_analysis_and_reporting'

In [19]:
# Change working directory to repository directory
repo_dir_path = os.path.join(drive_home, repo_dir_name)
os.chdir(repo_dir_path)
print(f'Current working directory set to: {os.path.abspath(os.curdir)}')

Current working directory set to: /content/drive/MyDrive/sdg_661_analysis_and_reporting


In [10]:
# Can now import the GSWE library
from GSWE_reporting import (extract_gswe, clip_basin_to_boundary,
                            get_gswe_paths, reproject_GSWE,
                            surface_water_extent)
from GSWE_reporting.GEE_extractor import GSWE_YEARLY

print(f'Using Asset ID: {GSWE_YEARLY}')

Using Asset ID: JRC/GSW1_3/YearlyHistory


In [ ]:
# Autheticate Earth Engine access
# Follow the link below to generate a verification code and enter in
# the box provided and hit return.
ee.Authenticate()

In [13]:
# Earth Engine requires initialisation
ee.Initialize()

## User Settings
These settings allow the user to specify the analysis time frame, where imagery is stored, and the names of the output files.

Note that since the working directory is set to `repo_dir_path` any location beginning with `.` are relative paths within that directory.

In [30]:
# Specify directory to store downloaded GSWE images. This directory must be a
# valid Google Drive path. If the directory does not exist it will be created.
# Default setting uses repo_dir_path/gswe_exports.
gswe_export_dir = './gswe_exports'

# Specify year range for analysis
start_yr = 1984  # <<<< Inclusive
end_yr = 2022  # <<<< Exclusive i.e. data in year == end_yr will be excluded

# Specify the directory to store reprojected images. Default setting uses
# repo_dir_path/Reprojected
reproj_dir = './Reprojected/'

# Output files
# By default these are saved in repo_dir_path. If such files already exist they
# will be overwritten!
water_extent_file = './water_extent_by_basin_by_year.csv'
water_extent_change_file = './water_extent_change.csv'

## Extracting GSWE data

In [14]:
# Create a bounding box geometry for the UK
geometry = ee.Geometry.Polygon(
        [[[-12.598544729822379, 61.78863494939058],
          [-12.598544729822379, 49.00174346741333],
          [3.749111520177621, 49.00174346741333],
          [3.749111520177621, 61.78863494939058]]])

In [ ]:
# Extract the GWSE imagery from Earth Engine.
assert start_yr is not None and end_yr is not None
extract_gswe(GSWE_YEARLY, geometry, start_yr, end_yr, gswe_export_dir)

This will start background tasks for downloading the imagery to your drive. You can check the status of these tasks at https://code.earthengine.google.com/tasks. Note that this may take some time.

---
## Preparing HydroBASIN boundary

This step can be done while the GSWE imagery is processing.

In [16]:
# Load hydrobasins
eu_hydrobasin = gpd.read_file('./boundaries/hybas_eu_lev06_v1c.shp')

# Load UK boundary
uk_boundary = gpd.read_file('./boundaries/Countries_December_2018_Boundaries_UK_BFC.shp') 

In [17]:
# Configure coordinate reference systems
target_crs = ('+proj=utm +zone=30 +ellps=WGS84 +datum=WGS84 +units=m +no_defs ')
osgb_proj4 = ('+proj=tmerc +lat_0=49 +lon_0=-2 +k=0.9996012717 +x_0=400000 +y_0=-100000 +ellps=airy +datum=OSGB36 +units=m +no_defs')
uk_boundary.crs = osgb_proj4  

In [18]:
# Clip the hydrobasins by the UK boundary
hydro_clipped = clip_basin_to_boundary(eu_hydrobasin, uk_boundary, target_crs)

union national boundary


In [22]:
# Clean and save file
hydro_clipped = hydro_clipped.drop(hydro_clipped.columns[1:-1], axis=1) 
hydro_clipped.to_file('./boundaries/UK_hydrobasin_UTM.shp')

---
## Preparing GSWE outputs

**Note:** Only continue once the imagery is available

The remaining cells of the notebook can be run without intervention by selecting `Runtime -> Run after` from the menu bar. However, users are encouraged to step through the cells to follow the process. Outputs will be created in the locations specified in USER SETTINGS above.

In [23]:
# Get paths to exported images
gsw_files = get_gswe_paths(gswe_export_dir, '.tif')
print(f'Found {len(gsw_files)} images.')

Found 37 images.


In [ ]:
# Reproject images to UTM for valid area calculations
# These files will be saved repo_dir_path/Reprojected/
reproject_GSWE(gsw_files, '+proj=utm +zone=30 +ellps=WGS84 +datum=WGS84 +units=m +no_defs ')

---
## Calculating Water Extent

In [29]:
# Get paths to the reprojected images
gsw_file_path_list = get_gswe_paths(reproj_dir, 'UTM.tif')
print(f'Found {len(gsw_file_path_list)} reprojected images.')

Found 0 reprojected images.


In [ ]:
# Read in the hydrobasin file created in section Preparing HydroBASIN boundary
hydro_basin = gpd.read_file('./boundaries/UK_hydrobasin_UTM.shp')

In [ ]:
# Calculate the water extent by type for each hydrobasin for each year
water_extent = surface_water_extent(gsw_file_path_list, hydro_basin)

In [ ]:
# Preview results
water_extent.head()

In [ ]:
# Calculate water extent as percentage of hydrobasin area
hydro_basin['area'] = hydro_basin.area / 1000000
water_extent = hydro_basin[['HYBAS_ID', 'area']].merge(water_extent, on='HYBAS_ID', how = 'left')
water_extent[['% Ephemeral', '% Seasonal', '% Permanent']] = water_extent[['Ephemeral', 'Seasonal', 'Permanent']].div(water_extent['area'], axis=0)*100

In [ ]:
# Preview results
water_extent.head()

,HYBAS_ID,area,Year,Ephemeral,Seasonal,Permanent,% Ephemeral,% Seasonal,% Permanent
0,2060048790,4569.73882,1984,2.810391,1.736887,3.386747,0.061500,0.038008,0.074112
1,2060048790,4569.73882,1985,2.571197,3.154850,3.879204,0.056266,0.069038,0.084889
2,2060048790,4569.73882,1986,3.083978,2.132416,3.560280,0.067487,0.046664,0.077910
3,2060048790,4569.73882,1987,2.735871,2.612366,3.802078,0.059869,0.057167,0.083201
4,2060048790,4569.73882,1988,2.205372,1.254853,3.036556,0.048260,0.027460,0.066449


### National Water Extent

In [ ]:
# Calculate national water extent
water_extent['HYBAS_ID'] = water_extent["HYBAS_ID"].apply(str)
water_type_by_year = water_extent.groupby(['Year'], as_index=False).sum() 

In [ ]:
# Preview Results - note that the percentages are recalculated in a later step
water_type_by_year.head()

,Year,area,Ephemeral,Seasonal,Permanent,% Ephemeral,% Seasonal,% Permanent
0,1984,244147.376651,325.378393,155.430802,1928.471933,4.889480,2.924425,48.755351
1,1985,244147.376651,287.807334,210.411378,1849.862814,9.629608,6.338746,54.330165
2,1986,244147.376651,284.834875,209.191962,1860.879253,5.149310,7.708008,57.129038
3,1987,244147.376651,245.206441,254.372389,2069.407310,4.520509,8.570212,59.629123
4,1988,244147.376651,245.224680,237.680452,2088.768415,7.842463,7.700770,56.639291


In [ ]:
# Visualise total UK area of water types by year
water_type_melt = pd.melt(water_type_by_year[water_type_by_year['Year']!='baseline'], id_vars=['Year'],
                          value_vars=['Ephemeral', 'Seasonal', 'Permanent'],
                          var_name='Type', value_name = 'Extent')

alt.Chart(water_type_melt).mark_bar(opacity=0.7).encode(
    x=alt.X('Year:N', title = 'Year'),
    y=alt.Y('sum(Extent)', title = 'Water Extent (km\N{SUPERSCRIPT TWO})'),
    color=alt.Color('Type',
        scale = alt.Scale(domain=['Permanent', 'Ephemeral', 'Seasonal'],
                          range=['#1f78b4', '#b2df8a', '#a6cee3']),
        title = 'Water Type'),
    order=alt.Order('sum(Type)',sort='ascending'),
    tooltip = ['Year:N', 
               alt.Tooltip('Extent:Q', title = 'Extent')]
).interactive()

alt.Chart(...)

In [ ]:
# Recalculate percentage breakdown
water_type_by_year['HYBAS_ID'] = 'Total'
water_type_by_year[['% Ephemeral', '% Seasonal', '% Permanent']] = water_type_by_year[['Ephemeral', 'Seasonal', 'Permanent']].div(water_type_by_year['area'], axis=0)*100
water_type_by_basin = water_extent.append(water_type_by_year, sort=False, ignore_index = True)

In [ ]:
# Export result
water_type_by_basin.to_csv(water_extent_file, header = True, index = False) 

---

## Percentage Extent Change

In [ ]:
# Load the water extent file
water_extent_df = pd.read_csv(water_extent_file)
water_extent_df = water_extent_df[['HYBAS_ID', 'Year', 'Ephemeral', 'Seasonal', 'Permanent']]

In [ ]:
# Calculate change from baseline period
cols = [ 'Water Type', 'HYBAS_ID','Year', 'Average Extent', '% Extent Change']
water_extent_change = pd.DataFrame(columns = cols)


def get_year_range(from_year):
    to_year = str(int(from_year)+4)
    return f"{from_year}-{to_year}"


for basin in water_extent_df['HYBAS_ID'].unique():
    hydrobasin_extent = water_extent_df[water_extent_df['HYBAS_ID']==basin]
    
    ## Seperate hydrobasin and baseline
    baseline = hydrobasin_extent[hydrobasin_extent['Year']=='baseline'].reset_index(drop=True)
    hydrobasin_extent = hydrobasin_extent[hydrobasin_extent['Year']!='baseline']  

    ## Create rolling average
    rolling_av = (hydrobasin_extent[['Ephemeral', 'Seasonal', 'Permanent']]
                  .rolling(5)
                  .mean()
                  .dropna()
                 .reset_index(drop=True))
    
    rolling_av['HYBAS_ID'] = basin
    rolling_av['Year'] = hydrobasin_extent.apply(lambda x :get_year_range(x["Year"]), axis=1).reset_index(drop=True)[:-4]
    
    ## Melt and index both rolling and baseline with water type
    rolling_melt = pd.melt(rolling_av, id_vars=['HYBAS_ID', 'Year'], value_vars=['Ephemeral', 'Seasonal', 'Permanent'],
                          var_name='Water Type', value_name='Average Extent').set_index(['Water Type', 'Year'])
    
    baseline_melt = pd.melt(baseline, id_vars=['HYBAS_ID', 'Year'], value_vars=['Ephemeral', 'Seasonal', 'Permanent'],
                          var_name='Water Type', value_name='Average Extent').set_index('Water Type')
    
    ## Calculate extent change - index will divide the rolling average by the correct water type
    extent_change = (((rolling_melt['Average Extent']-baseline_melt['Average Extent'])
                     /baseline_melt['Average Extent']*100).reset_index()['Average Extent'])
    rolling_melt = rolling_melt.reset_index()
    
    rolling_melt['% Extent Change'] = extent_change
    
    water_extent_change = pd.concat([water_extent_change, rolling_melt, baseline_melt.reset_index()])


In [ ]:
## we should exclude years where the GSWE data is marred by cloud cover
exclude_years = [str(i) for i in list(range(1991, 1998)) + list(range(2004,2009))]
water_extent_change = water_extent_change[~water_extent_change['Year'].str.contains('|'.join(exclude_years))]

In [ ]:
# Export result
water_extent_change.to_csv(water_extent_change_file, index = False)